# Import dependencies

In [ ]:

import requests
import sqlite3
import json
from datetime import datetime, timedelta
from time import sleep

# Set up SQLite database

In [ ]:

conn = sqlite3.connect("room_data.db")
cursor = conn.cursor()

# Create database structure
cursor.execute('''
CREATE TABLE IF NOT EXISTS rooms (
    id INTEGER PRIMARY KEY,
    floor INTEGER,
    number TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS sensors (
    room_id INTEGER,
    sensor_id INTEGER,
    name TEXT,
    available INTEGER,
    PRIMARY KEY (room_id, sensor_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS sensor_data (
    room_id INTEGER,
    timestamp TEXT,
    sensor TEXT,
    value REAL
)
''')

conn.commit()

# Authenticate and get token

In [ ]:

login_url = "http://leffe.science.uva.nl:8042/auth/login"
credentials = {
    "username": "******",
    "password": "******"
}

response = requests.post(login_url, json=credentials)
if response.status_code == 200:
    token = response.json()["access_token"]
    print("Authenticated successfully.")
else:
    raise Exception("Authentication failed:", response.text)

# Fetch room list from all floors (0–6)

In [ ]:

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}"
}

rooms = []

for floor in range(0, 7):
    rooms_url = f"http://leffe.science.uva.nl:8042/rooms?floor={floor}"
    rooms_response = requests.get(rooms_url, headers=headers)

    if rooms_response.status_code != 200:
        print(f"⚠️ Failed to get rooms on floor {floor}: {rooms_response.text}")
        continue

    floor_rooms = rooms_response.json()
    rooms.extend(floor_rooms)
    print(f"Retrieved {len(floor_rooms)} rooms from floor {floor}")

print(f"\n Total rooms retrieved: {len(rooms)}")


# Store room and sensor data

In [ ]:

for room in rooms:
    room_id = room["id"]
    floor = room["floor"]
    number = room["number"]

    cursor.execute("INSERT OR REPLACE INTO rooms (id, floor, number) VALUES (?, ?, ?)", (room_id, floor, number))

    for sensor in room["sensors"]:
        cursor.execute('''
        INSERT OR REPLACE INTO sensors (room_id, sensor_id, name, available)
        VALUES (?, ?, ?, ?)
        ''', (room_id, sensor["id"], sensor["name"], sensor["available"]))

conn.commit()
print("Room and sensor metadata stored.")

# Close open database connections

In [ ]:
conn.commit()
conn.close()

# Fetch all new data between dates (except exlcluded room ids)

In [ ]:
import aiohttp
import asyncio
import aiosqlite
from time import time
from datetime import datetime, timedelta


CONCURRENT_REQUESTS = 20 
DB_PATH = "room_data.db"
API_BASE_URL = "http://leffe.science.uva.nl:8042"

START_TIME = datetime(2025, 6, 3)
END_TIME = datetime(2025, 6, 4)
TIME_CHUNK = timedelta(hours=6)

# Manually excluded room IDs
EXCLUDED_ROOM_IDS = {50, 52, 53, 54, 55, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 81, 82, 83, 84, 85, 86, 96, 97, 98, 99, 100, 101, 102, 103, 106, 107, 108, 109, 110, 111, 112, 113, 114, 118, 119, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 260, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 312, 313, 314, 315, 316, 322, 323, 324, 325, 326}

def format_iso8601(dt):
    iso_str = dt.isoformat(timespec='milliseconds') + 'Z'
    return iso_str.replace(':', '%3A')

async def fetch_page(session, room_id, page, start_time=None, end_time=None):
    base_url = f"{API_BASE_URL}/rooms/{room_id}/data"
    params = {"page": page}
    
    if start_time and end_time:
        params.update({
            "startTime": format_iso8601(start_time),
            "endTime": format_iso8601(end_time)
        })
    
    url = f"{base_url}?{'&'.join(f'{k}={v}' for k,v in params.items())}"
    
    async with session.get(url) as response:
        if response.status != 200:
            print(f"Failed to fetch room {room_id} page {page}: {response.status}")
            return None
        return await response.json()

async def process_room_time_chunk(session, db, room, chunk_start, chunk_end):
    """Process a room for a specific time chunk"""
    room_id = room["id"]
    room_number = room["number"]
    
    if room_id in EXCLUDED_ROOM_IDS:
        return

    first_page = await fetch_page(session, room_id, 1, chunk_start, chunk_end)
    if not first_page:
        return

    total_pages = first_page.get("pagination", {}).get("totalPages", 1)
    results = first_page.get("results", [])

    await insert_results(db, room_id, results)

    if total_pages > 1:
        semaphore = asyncio.Semaphore(CONCURRENT_REQUESTS)

        async def fetch_and_store(page):
            async with semaphore:
                page_data = await fetch_page(session, room_id, page, chunk_start, chunk_end)
                if page_data:
                    await insert_results(db, room_id, page_data.get("results", []))

        tasks = [fetch_and_store(page) for page in range(2, total_pages + 1)]
        await asyncio.gather(*tasks)

async def process_room(session, db, room):
    current_start = START_TIME
    while current_start < END_TIME:
        current_end = min(current_start + TIME_CHUNK, END_TIME)
        await process_room_time_chunk(session, db, room, current_start, current_end)
        current_start = current_end
    print(f"Finished processing room {room['number']} (ID: {room['id']})")

async def insert_results(db, room_id, results):
    for record in results:
        timestamp = record.get("timestamp")
        for sensor, value in record.items():
            if sensor != "timestamp":
                await db.execute('''
                    INSERT INTO sensor_data (room_id, timestamp, sensor, value)
                    VALUES (?, ?, ?, ?)
                ''', (room_id, timestamp, sensor, value))

async def main_async():
    start_time = time()
    async with aiohttp.ClientSession(headers=headers) as session:
        async with aiosqlite.connect(DB_PATH) as db:
            await db.execute("PRAGMA journal_mode=WAL")
            
            for room in rooms:
                async with db.execute("BEGIN"):
                    await process_room(session, db, room)
                    await db.commit()
    
    total_time = time() - start_time
    print(f"\n All done in {total_time:.2f} seconds!")

# --- To start ---
await main_async()